# Assignment 2 Part 2 - Convolutional Neural Network


## (1) Designing an adjustable CNN


### Model


In [8]:
from math import floor
import torch
import torch.nn as nn


class DigitClassifier(nn.Module):
    def __init__(self,
                 kernel_size: int = 5,
                 stride: int = 1,
                 pooling_strategy=nn.MaxPool2d(2),
                 feature_maps: int = 16,
                 additional_conv: bool = False,
                 additional_fc: bool = False):
        super(DigitClassifier, self).__init__()

        self.kernel_size = kernel_size
        self.stride = stride
        self.pooling_strategy = pooling_strategy

        self.act = nn.ReLU()
        self.flat = nn.Flatten()

        self.conv1 = self.get_conv(1, feature_maps)
        self.conv2 = self.get_conv(feature_maps, 32)
        if additional_conv:
            self.conv3 = self.get_conv(32, 64)

        fc_input_size = self.calculate_fc_input_size(additional_conv)

        if additional_fc:
            self.fc1 = nn.Sequential(nn.Linear(fc_input_size, 1024), self.act)
            self.out = nn.Linear(1024, 10)
        else:
            self.out = nn.Linear(fc_input_size, 10)

    def get_conv(self, in_channels: int, out_channels: int) -> nn.Sequential:
        return nn.Sequential(
            nn.Conv2d(
                in_channels, out_channels, self.kernel_size, self.stride, 2),
            self.act,
            self.pooling_strategy
        )

    def calculate_fc_input_size(self, additional_conv: bool) -> int:
        with torch.no_grad():
            x = torch.zeros(1, 1, 28, 28)  # Assuming input size of 28x28
            x = self.conv1(x)
            x = self.conv2(x)
            if hasattr(self, 'conv3'):
                x = self.conv3(x)
            x = self.flat(x)
            fc_input_size = x.size(1)
        return fc_input_size

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        if (hasattr(self, 'conv3')):
            x = self.conv3(x)
        x = self.flat(x)
        if (hasattr(self, 'fc1')):
            x = self.fc1(x)
        x = self.out(x)
        return x


### Data


In [3]:
DATA_URL = "./data" # colab: "/content/2.2/data"


In [4]:
from typing import List, Tuple
from numpy import array, ndarray
from torch import Tensor
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms.functional import to_tensor
from csv import reader
from matplotlib import pyplot as plt


class Digit_Dataset(Dataset):
    def __init__(self, train: bool):
        super(Digit_Dataset, self).__init__()

        self.train = train

        filename = "train" if train else "test"
        self.data = self._load_tensors(f"{filename}x")
        self.labels = self._load_labels(f"{filename}y")

    def __len__(self) -> int:
        return len(self.data)

    def __getitem__(self, index: int) -> Tuple[ndarray, int]:
        return self.data[index], self.labels[index]

    def _load_tensors(self, filename: str) -> List[Tensor]:
        with open(f"{DATA_URL}/{filename}.csv", 'r') as csv_file:
            tensors = []
            for line in reader(csv_file):
                data = array(line, dtype='int')
                data = data.reshape((28, 28))
                tensors.append(to_tensor(data).float())
            return tensors

    def _load_labels(self, filename: str) -> List[int]:
        with open(f"{DATA_URL}/{filename}.csv", 'r') as csv_file:
            labels = []
            for line in reader(csv_file):
                labels.append(int(line.index('1')))
            return labels

    def display(self, index: int):
        img = self.data[index].reshape((28, 28))
        plt.imshow(img, cmap='gray')
        plt.show()

    def loader(self, batch_size: int) -> DataLoader:
        return DataLoader(self, batch_size=batch_size, shuffle=self.train)


### Training


In [5]:
from torch import optim


def train_model(model: DigitClassifier, data: DataLoader, num_epochs: int = 4, learning_rate: float = 0.001):
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), learning_rate)

    model.train()

    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(data):
            b_x, b_y = images, labels

            loss = loss_fn(model(b_x), b_y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            print(
                f"(Epoch {epoch+1} of {num_epochs} | Step {i+1} of {len(data)}) Loss: {loss.item():.4f}", end='\r')


### Evaluation


In [6]:
import torch


def eval_model(model: DigitClassifier, data: DataLoader) -> float:
    correct = 0
    total = 0

    model.eval()

    with torch.no_grad():
        for images, labels in data:
            outputs = model(images)
            _, predicted = torch.max(outputs, dim=1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    return correct / total


### Wrapping everything up


In [7]:
print("Loading data…")
dataset = Digit_Dataset(True)
test_dataset = Digit_Dataset(False)
print("Finished loading data.\n")


def train(model: DigitClassifier, num_epochs: int = 4, learning_rate: float = 0.001):
    print("Training model…")
    train_model(model, dataset.loader(
        100), num_epochs, learning_rate)
    print("\nFinished training.\n")


def eval(model: DigitClassifier):
    print("Evaluating model…")
    print(
        f"Training accuracy: {eval_model(model, dataset.loader(100))*100:.3f}%")
    print(
        f"Test accuracy: {eval_model(model, test_dataset.loader(500))*100:.3f}%")
